# Imports

In [76]:
#!conda install -c anaconda beautifulsoup4 --yes
#!conda install -c anaconda lxml --yes
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Packages loaded")

Packages loaded


# Load Data from web page into DataFrame

In [9]:
# Use requests to get the html content from url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

#create BeautifulSoup object to parse the page content
soup = BeautifulSoup(page.content, 'lxml')

#find the table in the content and assign to variable table
table = soup.find('table', class_='wikitable sortable')
# pd.read_html retuns a list of dataframes, so need to get the first element
dfs = pd.read_html(str(table))
df = dfs[0]
# Set column names
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

PostalCode           Borough               Neighborhood
0        M1A      Not assigned                        NaN
1        M2A      Not assigned                        NaN
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

# Remove records with Borough Not assigned

In [11]:
# Remove rows with Borough == Not assigned
neighborhoods = df[df.Borough != 'Not assigned']
neighborhoods = neighborhoods.reset_index(drop=True)
neighborhoods.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

# Verify the requirements

### It appears that postal codes don't appear double (any longer?) in the source on the wikipedia page. 
### Additionally, after removing records where Borough == Not assigned, there are no Neighborhoods left with value Not assigned.

In [49]:
# Examples to explore the dataset

# neighborhoods['Neighborhood'].unique()
# neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned']
# neighborhoods[neighborhoods['Postal Code'] == 'M9V']

# DataFrame Shape

In [12]:
neighborhoods.shape

(103, 3)

# Use CSV for LatLong as geocoder package is unreliable

In [8]:
!wget -q -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
# Create dataframe from csv
latlong_df = pd.read_csv('geospatial_data.csv')
# merge neighborhoods with the latlong data
neighborhoodslatlong = neighborhoods
neighborhoodslatlong = neighborhoodslatlong.join(latlong_df.set_index('Postal Code'), on='PostalCode')
neighborhoodslatlong
neighborhoodslatlong.head()

# neighborhoods_merged[neighborhoods_merged['PostalCode'] == 'M4M']

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

In [14]:
neighborhoodslatlong.shape

(103, 5)

# Neighborhood Analysis
### Get lat and long for Toronto

In [15]:

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# Create map of Toronto

In [103]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoodslatlong['Latitude'], neighborhoodslatlong['Longitude'], neighborhoodslatlong['Borough'], neighborhoodslatlong['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [50]:
# toronto_data = neighborhoodslatlong[neighborhoodslatlong['Borough'] == 'Manhattan'].reset_index(drop=True)
# toronto_data = neighborhoodslatlong.filter(like='Toronto')
toronto_data = neighborhoodslatlong[neighborhoodslatlong.Borough.str.contains('Toronto', case=False)]
toronto_data.head()

# toronto_data = neighborhoodslatlong
# toronto_data.head()

PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031

## Define foursquare vars

In [145]:
CLIENT_ID = '*******************************' # your Foursquare ID
CLIENT_SECRET = '*******************************' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *******************************
CLIENT_SECRET:*******************************


### Explore 1 neighborhood

In [39]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [146]:
radius = 500
LIMIT = 100


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=*******************************&client_secret=*******************************&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [41]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebba7fa95feaf001b0931e0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

### Function to get caegories of the vanues

In [42]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Explore the vanues for this neighborhood

In [46]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


### Function to get the nearby vanues

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get a list of all toronto venues

In [55]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

### Count venues grouped per neighborhood

In [56]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
Brockton, Parkdale Village, Exhibition Place                           24   
Business reply mail Processing Centre                                  16   
CN Tower, King and Spadina, Railway Lands, Harb...                     17   
Central Bay Street                                                     61   
Christie                                                               17   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        7   
Dufferin, Dovercourt Village                                           17   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          22   
India Bazaar, The Beaches West                                         18   
Kensington Market, Chinatown, Grange Park                              56   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               41   
Moore Park, Summerhill East                                             2   
North Toronto West                                                     22   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            34   
Regent Park, Harbourfront                                              46   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     38   
St. James Town                                                         77   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         94   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     17   
The Annex, North Midtown, Yorkville                                    24   
The Beaches                                                             6   
The Danforth West, Riverdale                                           42   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             56   
Brockton, Parkdale Village, Exhibition Place                            24   
Business reply mail Processing Centre                                   16   
CN Tower, King and Spadina, Railway Lands, Harb...                      17   
Central Bay Street                                                      61   
Christie                                                                17   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              35   
Davisville North                    

In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


In [59]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theme Restaurant  Toy / Game Store  Trail  \
0             0  ...                 0                 0      0   
1             0  ...                 0                 0      0   
2             0  ...                 0                 0      0   
3             0  ...                 0                 0      0   
4             0  ...                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  Women's Store  
0                      0         0            0              0  
1                      0         0            0              0  
2                      0         0            0              0  
3                      0         0            0              0  
4                      0         0            0              0  

[5 rows x 228 columns]

In [60]:
toronto_onehot.shape

(1606, 228)

In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2               Business reply mail Processing Centre     0.062500   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                  Central Bay Street     0.016393   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.025974   
7                      Commerce Court, Victoria Hotel     0.000000   
8                                          Davisville     0.000000   
9                                    Davisville North     0.000000   
10                       Dufferin, Dovercourt Village     0.000000   
11             First Canadian Place, Underground city     0.000000   
12                           Forest Hill North & West     0.000000   
13                           Garden District, Ryerson     0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000   
15                      High Park, The Junction South     0.000000   
16                     India Bazaar, The Beaches West     0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000   
18                                      Lawrence Park     0.000000   
19                           Little Portugal, Trinity     0.024390   
20                        Moore Park, Summerhill East     0.000000   
21                                 North Toronto West     0.045455   
22                             Parkdale, Roncesvalles     0.000000   
23        Queen's Park, Ontario Provincial Government     0.029412   
24                          Regent Park, Harbourfront     0.021739   
25                           Richmond, Adelaide, King     0.000000   
26                                           Rosedale     0.000000   
27                                           Roselawn     0.000000   
28                                 Runnymede, Swansea     0.026316   
29                                     St. James Town     0.000000   
30                        St. James Town, Cabbagetown     0.000000   
31                                     Stn A PO Boxes     0.010638   
32                                    Studio District     0.025000   
33  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
34                The Annex, North Midtown, Yorkville     0.000000   
35                                        The Beaches     0.000000   
36                       The Danforth West, Riverdale     0.023810   
37           Toronto Dominion Centre, Design Exchange     0.000000   
38                     University of Toronto, Harbord     0.029412   

    Afghan Restaurant   Airport  Airport Food Court  Airport Gate  \
0            0.000000  0.000000            0.000000      0.000000   
1            0.000000  0.000000            0.000000      0.000000   
2            0.000000  0.000000            0.000000      0.000000   
3            0.000000  0.058824            0.058824      0.058824   
4            0.000000  0.000000            0.000000      0.000000   
5            0.000000  0.000000            0.000000      0.000000   
6            0.012987  0.000000            0.000000      0.000000   
7            0.000000  0.000000            0.000000      0.000000   
8            0.000000  0.000000            0.000000      0.000000   
9            0.000000  0.000000            0.000000      0.000000   
10           0.000000  0.000000            0.000000      0.000000   
11           0.000000  0.000000            0.000000      0.000000   
12           0.000000  0.000000            0.000000      0.000000   
13           0.000000  0.000000            0.000000      0.000000   
14           0.000000  0.000000            0.000000      0.000000   
15           0.000000  0.000000            0.000000      0.000000   
16           0.000000  0.000000            0.000000      0.000000  

In [62]:
toronto_grouped.shape

(39, 228)

### Display the frequency of the top 5 of the venues

In [63]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1       Nightclub  0.12
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4       Pet Store  0.04


----Business reply mail Processing Centre----
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.18
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4      Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant

### Getting the most common venues

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2              Business reply mail Processing Centre           Yoga Studio   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar           Cheese Shop              Beer Bar   
1             Nightclub           Coffee Shop        Breakfast Spot   
2         Auto Workshop         Garden Center                Garden   
3        Airport Lounge      Airport Terminal         Boat or Ferry   
4    Italian Restaurant                  Café        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                Bakery    Seafood Restaurant                  Café   
1          Climbing Gym                   Bar                Bakery   
2  Fast Food Restaurant        Farmers Market    Light Rail Station   
3               Airport    Airport Food Court          Airport Gate   
4       Thai Restaurant        Ice Cream Shop          Burger Joint   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Restaurant                   Pub         Breakfast Spot  
1          Intersection            Restaurant     Italian Restaurant  
2            Comic Shop           Pizza Place          Burrito Place  
3           Coffee Shop                   Bar    Rental Car Location  
4           Salad Place                   Bar    Japanese Restaurant

# Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [141]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 3, 0, 0, 4, 0, 4, 5, 0, 1, 0,
       0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 4], dtype=int32)

In [142]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode           Borough                                 Neighborhood  \
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9         M5B  Downtown Toronto                     Garden District, Ryerson   
15        M5C  Downtown Toronto                               St. James Town   
19        M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
2   43.654260 -79.360636               0           Coffee Shop   
4   43.662301 -79.389494               0           Coffee Shop   
9   43.657162 -79.378937               0        Clothing Store   
15  43.651494 -79.375418               0                  Café   
19  43.676357 -79.293031               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
2                 Bakery                  Park                  Café   
4       Sushi Restaurant           Yoga Studio   Arts & Crafts Store   
9            Coffee Shop        Cosmetics Shop                  Café   
15           Coffee Shop          Cocktail Bar             Gastropub   
19     Health Food Store           Coffee Shop                   Pub   

        5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
2              Breakfast Spot            Restaurant               Theater   
4              Sandwich Place   Distribution Center   Japanese Restaurant   
9   Middle Eastern Restaurant       Bubble Tea Shop    Italian Restaurant   
15        American Restaurant        Cosmetics Shop    Italian Restaurant   
19                      Trail          Cupcake Shop          Dance Studio   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
2                    Pub        Chocolate Shop           Dessert Shop  
4     Italian Restaurant         Burrito Place               Beer Bar  
9             Restaurant   Japanese Restaurant   Fast Food Restaurant  
15                   Gym        Clothing Store               Beer Bar  
19         Deli / Bodega      Department Store                  Diner

# Creating Map with Clusters

In [143]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the clusters

## Cluster 1 - Coffee cluster The majority of venues falls in the coffee shop category or a category related to it

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
2    Downtown Toronto               0           Coffee Shop   
4    Downtown Toronto               0           Coffee Shop   
9    Downtown Toronto               0        Clothing Store   
15   Downtown Toronto               0                  Café   
19       East Toronto               0                  Park   
20   Downtown Toronto               0           Coffee Shop   
24   Downtown Toronto               0           Coffee Shop   
30   Downtown Toronto               0           Coffee Shop   
36   Downtown Toronto               0           Coffee Shop   
37       West Toronto               0                   Bar   
41       East Toronto               0      Greek Restaurant   
42   Downtown Toronto               0           Coffee Shop   
47       East Toronto               0  Fast Food Restaurant   
48   Downtown Toronto               0           Coffee Shop   
67    Central Toronto               0                  Park   
73    Central Toronto               0        Clothing Store   
75       West Toronto               0             Gift Shop   
79    Central Toronto               0          Dessert Shop   
81       West Toronto               0                  Café   
86    Central Toronto               0           Coffee Shop   
87   Downtown Toronto               0       Airport Service   
92   Downtown Toronto               0           Coffee Shop   
96   Downtown Toronto               0           Coffee Shop   
97   Downtown Toronto               0           Coffee Shop   
99   Downtown Toronto               0           Coffee Shop   
100      East Toronto               0           Yoga Studio   

    2nd Most Common Venue    3rd Most Common Venue 4th Most Common Venue  \
2                  Bakery                     Park                  Café   
4        Sushi Restaurant              Yoga Studio   Arts & Crafts Store   
9             Coffee Shop           Cosmetics Shop                  Café   
15            Coffee Shop             Cocktail Bar             Gastropub   
19      Health Food Store              Coffee Shop                   Pub   
20           Cocktail Bar              Cheese Shop              Beer Bar   
24     Italian Restaurant                     Café        Sandwich Place   
30                   Café               Restaurant                   Gym   
36               Aquarium                     Café                 Hotel   
37             Restaurant         Asian Restaurant           Men's Store   
41     Italian Restaurant              Coffee Shop            Restaurant   
42                   Café                    Hotel            Restaurant   
47             Steakhouse        Fish & Chips Shop                   Pub   
48                   Café               Restaurant                 Hotel   
67      Food & Drink Shop           Breakfast Spot        Sandwich Place   
73            Coffee Shop     Fast Food Restaurant                  Park   
75         Breakfast Spot              Coffee Shop      Cuban Restaurant   
79         Sandwich Place                      Gym           Coffee Shop   
81            Coffee Shop                      Pub           Pizza Place   
86                    Pub  Health & Beauty Service    Light Rail Station   
87         Airport Lounge         Airport Terminal         Boat or Ferry   
92                   Café       Seafood Restaurant              Beer Bar   
96             Restaurant                     Café                   Pub   
97                   Café                      Gym   Japanese Restaurant   
99    Japanese Restaurant         Sushi Restaurant            Restaurant   
100         Auto Workshop            Garden Center                Garden   

         5th Most Common Venue          6th Most Common Venue  \
2               Breakfast Spot                     Restaurant   
4               Sandwich Place            Distribution Center   
9    Middle Eastern Restaurant                Bubble Tea Shop   

## Cluster 2 - Lack of gyms in Toronto?

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
83  Central Toronto               1                   Gym   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
83            Playground         Women's Store          Cupcake Shop   

          5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
83  Eastern European Restaurant            Donut Shop      Doner Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
83               Dog Run   Distribution Center         Discount Store

## Cluster 3

In [130]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
62  Central Toronto               2                Garden   

   2nd Most Common Venue 3rd Most Common Venue        4th Most Common Venue  \
62         Women's Store          Cupcake Shop  Eastern European Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
62            Donut Shop      Doner Restaurant               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
62   Distribution Center        Discount Store                  Diner

## Cluster 4

In [131]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
68  Central Toronto               3         Jewelry Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
68                 Trail    Mexican Restaurant      Sushi Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
68         Women's Store          Dance Studio            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
68      Doner Restaurant               Dog Run    Distribution Center

## Cluster 5 - It's all about the food here! Cafe is the most common venue category

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
25  Downtown Toronto               4         Grocery Store   
31      West Toronto               4              Pharmacy   
43      West Toronto               4                  Café   
54      East Toronto               4                  Café   
69      West Toronto               4       Thai Restaurant   
74   Central Toronto               4        Sandwich Place   
80  Downtown Toronto               4                  Café   
84  Downtown Toronto               4                  Café   

   2nd Most Common Venue 3rd Most Common Venue      4th Most Common Venue  \
25                  Café                  Park                Candy Store   
31                Bakery                  Park                    Brewery   
43             Nightclub           Coffee Shop             Breakfast Spot   
54           Coffee Shop                Bakery                    Brewery   
69                  Café    Mexican Restaurant                       Park   
74                  Café           Coffee Shop  Middle Eastern Restaurant   
80    Italian Restaurant            Restaurant                        Bar   
84           Coffee Shop    Mexican Restaurant      Vietnamese Restaurant   

   5th Most Common Venue 6th Most Common Venue          7th Most Common Venue  \
25    Italian Restaurant                 Diner                     Restaurant   
31                  Café                   Bar                           Bank   
43          Climbing Gym                   Bar                         Bakery   
54   American Restaurant             Gastropub             Seafood Restaurant   
69                Bakery           Flea Market      Cajun / Creole Restaurant   
74              Pharmacy     Indian Restaurant                      BBQ Joint   
80   Japanese Restaurant             Bookstore                         Bakery   
84                Bakery           Gaming Cafe  Vegetarian / Vegan Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
25            Baby Store    Athletics & Sports            Coffee Shop  
31           Supermarket                  Pool            Art Gallery  
43          Intersection            Restaurant     Italian Restaurant  
54             Bookstore        Sandwich Place            Cheese Shop  
69             Speakeasy  Fast Food Restaurant    Arts & Crafts Store  
74        History Museum           Cheese Shop           Burger Joint  
80              Beer Bar            Beer Store         Sandwich Place  
84                   Bar          Dessert Shop    Japanese Restaurant

## Cluster 6

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
61  Central Toronto               5              Bus Line   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
61           Swim School                  Park           College Gym   

   5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
61    College Auditorium  Eastern European Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
61      Doner Restaurant               Dog Run    Distribution Center

## Cluster 7

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
91  Downtown Toronto               6                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
91                 Trail            Playground              Creperie   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
91            Donut Shop      Doner Restaurant               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
91   Distribution Center        Discount Store                  Diner